# References

This notebook contains analysis of certificate references in Common Criteria certificates.

The notebook has two parts, an analysis part and a network visualization part.
But first some common initialization and data loading.

In [ ]:
import networkx as nx
import networkx.algorithms.community as nx_comm
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from sec_certs.dataset.cc import CCDataset
import pandas as pd
import seaborn as sns
import numpy as np
from pysankey import sankey

%matplotlib inline

matplotlib.use("pgf")
sns.set_theme(style='white')
plt.rcParams["axes.linewidth"] = 0.5
plt.rcParams["legend.fontsize"] = 6.5
plt.rcParams["xtick.labelsize"] = 8
plt.rcParams["ytick.labelsize"] = 8
plt.rcParams["ytick.left"] = True
plt.rcParams['ytick.major.size'] = 5
plt.rcParams['ytick.major.width'] = 0.5
plt.rcParams['ytick.major.pad'] = 0
plt.rcParams["xtick.bottom"] = True
plt.rcParams['xtick.major.size'] = 5
plt.rcParams['xtick.major.width'] = 0.5
plt.rcParams['xtick.major.pad'] = 0
plt.rcParams["pgf.texsystem"] = "pdflatex"
plt.rcParams["font.family"] = "serif"
plt.rcParams["text.usetex"] = True
plt.rcParams["pgf.rcfonts"] = False
plt.rcParams["axes.titlesize"] = 8
plt.rcParams["legend.handletextpad"] = 0.3
plt.rcParams['lines.markersize'] = 4
plt.rcParams['savefig.pad_inches'] = 0.01
sns.set_palette("deep")

#plt.style.use("seaborn-whitegrid")
#sns.set_palette("deep")
#sns.set_context("notebook")  # Set to "paper" for use in paper :)

#plt.rcParams['figure.figsize'] = (10, 6)

In [ ]:
# Initialize
dset = CCDataset.from_web_latest()

In [ ]:
df = dset.to_pandas()
df_id_rich = df.loc[df.cert_id.notnull()].copy()

## Reference analysis


### Count numbers of reference-rich certificates

- From the numbers follows that whenever a certificate is directly referencing some else, it also indirectly references some else
- We have more outgoing references than ingoing references, which kinda makes sense. You don't have to be aware that some other cert references you

In [ ]:
df["has_outgoing_direct_references"] = df.directly_referencing.notnull()
df["has_incoming_direct_references"] = df.directly_referenced_by.notnull()
df["has_outgoing_indirect_references"] = df.indirectly_referencing.notnull()
df["has_incoming_indirect_references"] = df.indirectly_referenced_by.notnull()

#df.loc[:, ["directly_referenced_by", "indirectly_referenced_by", "directly_referencing", "indirectly_referencing"]].notnull().describe()

print(f"\\newcommand{{\\numCcAllDirectReferencing}}{{{df.has_outgoing_direct_references.sum()}}}")
print(f"\\newcommand{{\\numCcAllNotDirectReferencing}}{{{len(df) - df.has_outgoing_direct_references.sum()}}}")

df_id_rich["has_outgoing_direct_references"] = df_id_rich.directly_referencing.notnull()
df_id_rich["has_incoming_direct_references"] = df_id_rich.directly_referenced_by.notnull()
df_id_rich["has_outgoing_indirect_references"] = df_id_rich.indirectly_referencing.notnull()
df_id_rich["has_incoming_indirect_references"] = df_id_rich.indirectly_referenced_by.notnull()

print(f"\\newcommand{{\\numCcWithIdDirectReferencing}}{{{df_id_rich.has_outgoing_direct_references.sum()}}}")
print(f"\\newcommand{{\\numCcWithIdNotDirectReferencing}}{{{len(df_id_rich) - df_id_rich.has_outgoing_direct_references.sum()}}}")

#df_id_rich.loc[:, ["directly_referenced_by", "indirectly_referenced_by", "directly_referencing", "indirectly_referencing"]].notnull().describe()

In [ ]:
print(f"\\newcommand{{\\numCCActiveDirectReferencing}}{{{df_id_rich.loc[df_id_rich.status == 'active'].has_outgoing_direct_references.sum()}}}")

archived_cert_id_list = set(df_id_rich[df_id_rich.status == "archived"].cert_id)
def contains_archived_cert_reference(referencing):
    if pd.isnull(referencing):
        return False
    return bool(archived_cert_id_list.intersection(referencing))
print(f"\\newcommand{{\\numCCActiveDirectReferencingArchived}}{{{df_id_rich[df_id_rich.status == 'active'].directly_referencing.apply(contains_archived_cert_reference).sum()}}}")

### Plot direct references per category

In [ ]:
figure, axes = plt.subplots(1, 2)
figure.set_size_inches(16, 10)
figure.set_tight_layout(True)

col_to_depict = ["has_outgoing_direct_references", "has_incoming_direct_references"]

for index, col in enumerate(col_to_depict):
    countplot = sns.countplot(data=df, x="category", hue=col, ax=axes[index])
    countplot.set(
        xlabel="Category",
        ylabel="Outgoing direct references",
        title=f"Countplot of {' '.join(col.split('_'))}",
    )
    countplot.tick_params(axis="x", rotation=90)
    countplot.legend(title=' '.join(col.split('_')), bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.0)


In [ ]:
cert_id_to_category_mapping = dict(zip(df.cert_id, df.category))
cert_id_to_category_mapping[np.NaN] = "No references"

exploded = df_id_rich.loc[:, ["category", "directly_referencing"]].explode("directly_referencing")
exploded["ref_category"] = exploded.directly_referencing.map(lambda x: cert_id_to_category_mapping[x] if pd.notnull(x) else np.nan)
exploded = exploded.loc[exploded.ref_category.notnull()]

exploded_with_refs = exploded.loc[exploded.ref_category != "No references"]
print(f"\\newcommand{{\\numCCDirectRefsSameCategory}}{{{(exploded_with_refs.category == exploded_with_refs.ref_category).sum()}}}")
print(f"\\newcommand{{\\numCCDirectRefsOtherCategory}}{{{(exploded_with_refs.category != exploded_with_refs.ref_category).sum()}}}")
print(f"\\newcommand{{\\numCCDirectRefs}}{{{len(exploded_with_refs)}}}")
print(f"\\newcommand{{\\numCCDirectRefsFromSmartcards}}{{{(exploded_with_refs.category == 'ICs, Smart Cards and Smart Card-Related Devices and Systems').sum()}}}")

all_categories = set(exploded.category.unique()) | set(exploded.ref_category.unique())
colors = list(sns.color_palette("hls", len(all_categories), as_cmap=False).as_hex())
color_dict = dict(zip(all_categories, colors))

figure, axes = plt.subplots(1, 1)
figure.set_size_inches(24, 10)
figure.set_tight_layout(True)

sankey(exploded.category, exploded.ref_category, colorDict=color_dict, leftLabels=list(exploded.category.unique()), rightLabels=list(exploded.ref_category.unique()), fontsize=12, ax=axes)

figure.savefig("category_references.pdf", bbox_inches="tight")
figure.savefig("category_references.pgf", bbox_inches="tight")
plt.close(figure)

### Plot direct references per scheme

In [ ]:
figure, axes = plt.subplots(1, 2)
figure.set_size_inches(14, 4)
figure.set_tight_layout(True)

col_to_depict = ["has_outgoing_direct_references", "has_incoming_direct_references"]

for index, col in enumerate(col_to_depict):
    countplot = sns.countplot(data=df, x="scheme", hue=col, ax=axes[index])
    countplot.set(
        xlabel="Category",
        ylabel="Outgoing direct references",
        title=f"Countplot of {' '.join(col.split('_'))}",
    )
    countplot.tick_params(axis="x", rotation=90)
    countplot.legend(title=' '.join(col.split('_')), bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.0)

### Number of certificates referencing archived certificates

In [ ]:
def references_archived_cert(references):
    if pd.isnull(references):
        return False

    return any([x in cert_ids] for x in references)

cert_ids = set(df.loc[((df.cert_id.notnull()) & (df.status == "archived")), "cert_id"].tolist())
df["references_archived_cert"] = df.directly_referenced_by.map(references_archived_cert)

print(f"Number of certificates that reference some archived certificate: {df.loc[df.references_archived_cert].shape[0]}")

col_to_depict = ["category", "scheme"]

figure, axes = plt.subplots(1, 2)
figure.set_size_inches(14, 8)
figure.set_tight_layout(True)

for index, col in enumerate(col_to_depict):
    countplot = sns.countplot(data=df, x=col, hue="references_archived_cert", ax=axes[index])
    countplot.set(
        xlabel=col,
        ylabel="Outgoing direct references",
        title="Countplot of certificates that reference some archived certificate",
    )
    countplot.tick_params(axis="x", rotation=90)
    countplot.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.0)

### Count scheme references

In [ ]:
cert_id_to_scheme_mapping = dict(zip(df.cert_id, df.scheme))

df_ref_rich = df_id_rich.loc[df.directly_referencing.notnull()]
exploded = df_ref_rich.loc[:, ["scheme", "directly_referencing"]].explode("directly_referencing")

exploded["ref_scheme"] = exploded.directly_referencing.map(cert_id_to_scheme_mapping)
exploded = exploded.loc[exploded.ref_scheme.notnull()]

all_schemes = set(exploded.scheme.unique()) | set(exploded.ref_scheme.unique())
colors = list(sns.color_palette("hls", len(all_schemes), as_cmap=False).as_hex())
color_dict = dict(zip(all_schemes, colors))

figure, axes = plt.subplots(1, 1)
figure.set_size_inches(4, 4)
figure.set_tight_layout(True)

sankey(exploded.scheme, exploded.ref_scheme, colorDict=color_dict, leftLabels=list(exploded.scheme.unique()), rightLabels=list(exploded.ref_scheme.unique()), fontsize=7, ax=axes)

figure.savefig("scheme_references.pdf", bbox_inches="tight")
figure.savefig("scheme_references.pgf", bbox_inches="tight")
plt.close(figure)

In [ ]:
print(f"\\newcommand{{\\numCCUSReferencing}}{{{len(df_id_rich.loc[(df_id_rich.scheme == 'US') & (df_id_rich.directly_referencing.notnull())])}}}")
print(f"\\newcommand{{\\numCCUS}}{{{len(df_id_rich.loc[(df_id_rich.scheme == 'US')])}}}")

### Temporal evolution of references

Shows plot with relative number of certificates for a given year that reference some other certificate

In [ ]:
df_temporal = df.loc[df.year_from < 2022].groupby(["year_from"])["directly_referencing"].count().reset_index().set_index("year_from")
n_issued_certs = df.groupby("year_from").name.count().reset_index().rename(columns={"name": "n_certs"}).set_index("year_from")
df_temporal.directly_referencing = 100 * df_temporal.directly_referencing / n_issued_certs.n_certs

line = sns.lineplot(data=df_temporal, x="year_from", y="directly_referencing")
line.yaxis.set_major_formatter(mtick.PercentFormatter())

### Cross references

In [ ]:
# Plotting w.r.t. scheme and category (both are interesting)

## Reference network visualization

In [ ]:
certs_with_ids = {cert.heuristics.cert_id: cert for cert in dset if cert.heuristics.cert_id}

print(f"Certificates in dataset: {len(dset)}")
print(f"Certificates with extracted IDs: {len(certs_with_ids)}")

### Certificate report references

In [ ]:
refs_cr = nx.DiGraph()
for cert_id, cert in certs_with_ids.items():
    refs_cr.add_node(cert_id, cert=cert)
for cert_id, cert in certs_with_ids.items():
    if cr_refs := cert.heuristics.report_references.directly_referencing:
        for ref_id in cr_refs:
            if ref_id in certs_with_ids:
                refs_cr.add_edge(cert_id, ref_id, type=("cr",))
print(f"References in certificate reports: {len(refs_cr.edges)}")

### Security target references

In [ ]:
refs_st = nx.DiGraph()
for cert_id, cert in certs_with_ids.items():
    refs_st.add_node(cert_id, cert=cert)
for cert_id, cert in certs_with_ids.items():
    if st_refs := cert.heuristics.st_references.directly_referencing:
        for ref_id in st_refs:
            if ref_id in certs_with_ids:
                refs_st.add_edge(cert_id, ref_id, type=("st",))
print(f"References in security targets: {len(refs_st.edges)}")

### Combined references

In [ ]:
refs = nx.DiGraph()
for cert_id, cert in certs_with_ids.items():
    refs.add_node(cert_id, cert=cert)

for cert_id, cert in certs_with_ids.items():
    cr_refs = cert.heuristics.report_references.directly_referencing
    st_refs = cert.heuristics.st_references.directly_referencing
    cr_refs = set(cr_refs) if cr_refs is not None else set()
    st_refs = set(st_refs) if st_refs is not None else set()
    both = cr_refs.union(st_refs)
    for ref in both:
        if ref not in certs_with_ids:
            continue
        if ref in cr_refs and ref not in st_refs:
            refs.add_edge(cert_id, ref, type=("cr", ))
        elif ref in st_refs and ref not in cr_refs:
            refs.add_edge(cert_id, ref, type=("st", ))
        else:
            refs.add_edge(cert_id, ref, type=("cr", "st"))
print(f"Combined references (not double counted): {len(refs.edges)}")

### Certificate overview
Enter the certificate you are interested in below and see its reference graph component.

In [ ]:
cert_id = "ANSSI-CC-2019/02"

In [ ]:
cert = certs_with_ids.get(cert_id)
if cert is None:
    print(f"Certificate with id {cert_id} is not present in the dataset.")

for component in nx.weakly_connected_components(refs):
    if cert_id in component:
        break

view = nx.subgraph_view(refs, lambda node: node in component)
print(f"Certificate with id {cert_id}:")
print(f" - is in a component with {len(view.nodes)} certificates and {len(view.edges)} references.")
print(f" - references {list(view[cert_id].keys())}")
print(f" - is referenced by {list(view.predecessors(cert_id))}")
print(f" - its page is at https://seccerts.org/cc/{cert.dgst}/")

In [ ]:
nx.draw(view, pos=nx.planar_layout(view), with_labels=True)

## Some graph metrics
From <https://dataground.io/2021/09/29/simple-graph-metrics-networkx-for-beginners/> and
<https://theslaps.medium.com/centrality-metrics-via-networkx-python-e13e60ba2740>.
Also good <https://www.geeksforgeeks.org/network-centrality-measures-in-a-graph-using-networkx-python/>

In [ ]:
print(f"Density = {nx.density(refs)}")
print(f"Transitivity = {nx.transitivity(refs)}")

In [ ]:
print("Degree centrality <Popularity> (top 20):")
degree_centrality_vals = [(node, val) for node, val in nx.degree_centrality(refs).items()]
degree_centrality_vals.sort(key=lambda pair: pair[1], reverse=True)
for pair in degree_centrality_vals[:20]:
    print(f"\t{pair[0]} = {pair[1]}")

In [ ]:
print("Eigenvector centrality <Influence> (top 20):")
eigenvector_centrality_vals = [(node, val) for node, val in nx.eigenvector_centrality(refs).items()]
eigenvector_centrality_vals.sort(key=lambda pair: pair[1], reverse=True)
for pair in eigenvector_centrality_vals[:20]:
    print(f"\t{pair[0]} = {pair[1]}")

In [ ]:
print("Closeness centrality <Centralness> (top 20):")
closeness_centrality_vals = [(node, val) for node, val in nx.closeness_centrality(refs).items()]
closeness_centrality_vals.sort(key=lambda pair: pair[1], reverse=True)
for pair in closeness_centrality_vals[:20]:
    print(f"\t{pair[0]} = {pair[1]}")

In [ ]:
print("Betweenness centrality <Bridge> (top 20):")
betweenness_centrality_vals = [(node, val) for node, val in nx.betweenness_centrality(refs).items()]
betweenness_centrality_vals.sort(key=lambda pair: pair[1], reverse=True)
for pair in betweenness_centrality_vals[:20]:
    print(f"\t{pair[0]} = {pair[1]}")

In [ ]:
component_lengths = list(filter(lambda comp_len: comp_len > 1, map(len, nx.weakly_connected_components(refs))))
component_lengths.sort(reverse=True)
print(component_lengths)

In [ ]:
big_boy = refs.subgraph(max(nx.weakly_connected_components(refs), key=len))
communities = list(nx_comm.greedy_modularity_communities(big_boy))
print(len(communities))

In [ ]:
for com in communities:
    for i in sorted(com):
        print(f"\t{i}")

In [ ]:
import logging
import warnings
from collections import defaultdict
from typing import Any, Dict, List, Optional, Set, Tuple, Union

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from numpy import float64, ndarray
from pandas.core.frame import DataFrame
from pandas.core.series import Series

class PySankeyException(Exception):
    """Generic PySankey Exception."""


class NullsInFrame(PySankeyException):
    pass


class LabelMismatch(PySankeyException):
    pass

LOGGER = logging.getLogger(__name__)


def check_data_matches_labels(
    labels: Union[List[str], Set[str]], data: Series, side: str
) -> None:
    """Check whether data matches labels.
    Raise a LabelMismatch Exception if not."""
    if len(labels) > 0:
        if isinstance(data, list):
            data = set(data)
        if isinstance(data, pd.Series):
            data = set(data.unique().tolist())
        if isinstance(labels, list):
            labels = set(labels)
        if labels != data:
            msg = "\n"
            if len(labels) <= 20:
                msg = "Labels: " + ",".join(labels) + "\n"
            if len(data) < 20:
                msg += "Data: " + ",".join(data)
            raise LabelMismatch(f"{side} labels and data do not match.{msg}")


def sankey(
    left: Union[List, ndarray, Series],
    right: Union[ndarray, Series],
    leftWeight: Optional[ndarray] = None,
    rightWeight: Optional[ndarray] = None,
    colorDict: Optional[Dict[str, str]] = None,
    leftLabels: Optional[List[str]] = None,
    rightLabels: Optional[List[str]] = None,
    aspect: int = 4,
    rightColor: bool = False,
    fontsize: int = 14,
    figureName: Optional[str] = None,
    closePlot: bool = False,
    figSize: Optional[Tuple[int, int]] = None,
    ax: Optional[Any] = None,
) -> Any:
    """
    Make Sankey Diagram showing flow from left-->right
    Inputs:
        left = NumPy array of object labels on the left of the diagram
        right = NumPy array of corresponding labels on the right of the diagram
            len(right) == len(left)
        leftWeight = NumPy array of weights for each strip starting from the
            left of the diagram, if not specified 1 is assigned
        rightWeight = NumPy array of weights for each strip starting from the
            right of the diagram, if not specified the corresponding leftWeight
            is assigned
        colorDict = Dictionary of colors to use for each label
            {'label':'color'}
        leftLabels = order of the left labels in the diagram
        rightLabels = order of the right labels in the diagram
        aspect = vertical extent of the diagram in units of horizontal extent
        rightColor = If true, each strip in the diagram will be be colored
                    according to its left label
        figSize = tuple setting the width and height of the sankey diagram.
            Defaults to current figure size
        ax = optional, matplotlib axes to plot on, otherwise uses current axes.
    Output:
        ax : matplotlib Axes
    """
    ax, leftLabels, leftWeight, rightLabels, rightWeight = init_values(
        ax,
        closePlot,
        figSize,
        figureName,
        left,
        leftLabels,
        leftWeight,
        rightLabels,
        rightWeight,
    )
    plt.rc("text", usetex=False)
    plt.rc("font", family="serif")
    data_frame = _create_dataframe(left, leftWeight, right, rightWeight)
    # Identify all labels that appear 'left' or 'right'
    all_labels = pd.Series(
        np.r_[data_frame.left.unique(), data_frame.right.unique()]
    ).unique()
    LOGGER.debug("Labels to handle : %s", all_labels)
    leftLabels, rightLabels = identify_labels(data_frame, leftLabels, rightLabels)
    colorDict = create_colors(all_labels, colorDict)  # type: ignore
    ns_l, ns_r = determine_widths(data_frame, leftLabels, rightLabels)
    # Determine positions of left label patches and total widths
    leftWidths, topEdge = _get_positions_and_total_widths(
        data_frame, leftLabels, "left"
    )
    # Determine positions of right label patches and total widths
    rightWidths, topEdge = _get_positions_and_total_widths(
        data_frame, rightLabels, "right"
    )
    # Total vertical extent of diagram
    xMax = topEdge / aspect
    draw_vertical_bars(
        ax,
        colorDict,  # type: ignore
        fontsize,
        leftLabels,
        leftWidths,
        rightLabels,
        rightWidths,
        xMax,  # type: ignore
    )
    plot_strips(
        ax,
        colorDict,  # type: ignore
        data_frame,
        leftLabels,
        leftWidths,
        ns_l,
        ns_r,
        rightColor,
        rightLabels,
        rightWidths,
        xMax,
    )
    if figSize is not None:
        plt.gcf().set_size_inches(figSize)
    save_image(figureName)
    if closePlot:
        plt.close()
    return ax


def save_image(figureName: Optional[str]) -> None:
    if figureName is not None:
        file_name = f"{figureName}.png"
        plt.savefig(file_name, bbox_inches="tight", dpi=150)
        LOGGER.info("Sankey diagram generated in '%s'", file_name)


def identify_labels(
    dataFrame: DataFrame, leftLabels: List[str], rightLabels: List[str]
) -> Tuple[ndarray, ndarray]:
    # Identify left labels
    if len(leftLabels) == 0:
        leftLabels = pd.Series(dataFrame.left.unique()).unique()
    else:
        check_data_matches_labels(leftLabels, dataFrame["left"], "left")
    # Identify right labels
    if len(rightLabels) == 0:
        rightLabels = pd.Series(dataFrame.right.unique()).unique()
    else:
        check_data_matches_labels(rightLabels, dataFrame["right"], "right")
    return leftLabels, rightLabels


def init_values(
    ax: Optional[Any],
    closePlot: bool,
    figSize: Optional[Tuple[int, int]],
    figureName: Optional[str],
    left: Union[List, ndarray, Series],
    leftLabels: Optional[List[str]],
    leftWeight: Optional[ndarray],
    rightLabels: Optional[List[str]],
    rightWeight: Optional[ndarray],
) -> Tuple[Any, List[str], ndarray, List[str], ndarray]:
    deprecation_warnings(closePlot, figSize, figureName)
    if ax is None:
        ax = plt.gca()
    if leftWeight is None:
        leftWeight = []
    if rightWeight is None:
        rightWeight = []
    if leftLabels is None:
        leftLabels = []
    if rightLabels is None:
        rightLabels = []
    # Check weights
    if len(leftWeight) == 0:
        leftWeight = np.ones(len(left))
    if len(rightWeight) == 0:
        rightWeight = leftWeight
    return ax, leftLabels, leftWeight, rightLabels, rightWeight


def deprecation_warnings(
    closePlot: bool, figSize: Optional[Tuple[int, int]], figureName: Optional[str]
) -> None:
    warn = []
    if figureName is not None:
        msg = "use of figureName in sankey() is deprecated"
        warnings.warn(msg, DeprecationWarning)
        warn.append(msg[7:-14])
    if closePlot is not False:
        msg = "use of closePlot in sankey() is deprecated"
        warnings.warn(msg, DeprecationWarning)
        warn.append(msg[7:-14])
    if figSize is not None:
        msg = "use of figSize in sankey() is deprecated"
        warnings.warn(msg, DeprecationWarning)
        warn.append(msg[7:-14])
    if warn:
        LOGGER.warning(
            " The following arguments are deprecated and should be removed: %s",
            ", ".join(warn),
        )


def determine_widths(
    dataFrame: DataFrame, leftLabels: ndarray, rightLabels: ndarray
) -> Tuple[Dict, Dict]:
    # Determine widths of individual strips
    ns_l: Dict = defaultdict()
    ns_r: Dict = defaultdict()
    for leftLabel in leftLabels:
        left_dict = {}
        right_dict = {}
        for rightLabel in rightLabels:
            left_dict[rightLabel] = dataFrame[
                (dataFrame.left == leftLabel) & (dataFrame.right == rightLabel)
            ].leftWeight.sum()
            right_dict[rightLabel] = dataFrame[
                (dataFrame.left == leftLabel) & (dataFrame.right == rightLabel)
            ].rightWeight.sum()
        ns_l[leftLabel] = left_dict
        ns_r[leftLabel] = right_dict
    return ns_l, ns_r


def draw_vertical_bars(
    ax: Any,
    colorDict: Union[Dict[str, Tuple[float, float, float]], Dict[str, str]],
    fontsize: int,
    leftLabels: ndarray,
    leftWidths: Dict,
    rightLabels: ndarray,
    rightWidths: Dict,
    xMax: float64,
) -> None:
    # Draw vertical bars on left and right of each  label's section & print label
    for leftLabel in leftLabels:
        ax.fill_between(
            [-0.02 * xMax, 0],
            2 * [leftWidths[leftLabel]["bottom"]],
            2 * [leftWidths[leftLabel]["bottom"] + leftWidths[leftLabel]["left"]],
            color=colorDict[leftLabel],
            alpha=0.99,
        )
        ax.text(
            -0.05 * xMax,
            leftWidths[leftLabel]["bottom"] + 0.5 * leftWidths[leftLabel]["left"],
            leftLabel,
            {"ha": "right", "va": "center"},
            fontsize=fontsize,
        )
    for rightLabel in rightLabels:
        ax.fill_between(
            [xMax, 1.02 * xMax],
            2 * [rightWidths[rightLabel]["bottom"]],
            2 * [rightWidths[rightLabel]["bottom"] + rightWidths[rightLabel]["right"]],
            color=colorDict[rightLabel],
            alpha=0.99,
        )
        ax.text(
            1.05 * xMax,
            rightWidths[rightLabel]["bottom"] + 0.5 * rightWidths[rightLabel]["right"],
            rightLabel,
            {"ha": "left", "va": "center"},
            fontsize=fontsize,
        )


def create_colors(
    allLabels: ndarray, colorDict: Optional[Dict[str, str]]
) -> Union[Dict[str, Tuple[float, float, float]], Dict[str, str]]:
    # If no colorDict given, make one
    if colorDict is None:
        colorDict = {}
        palette = "hls"
        colorPalette = sns.color_palette(palette, len(allLabels))
        for i, label in enumerate(allLabels):
            colorDict[label] = colorPalette[i]
    else:
        missing = [label for label in allLabels if label not in colorDict.keys()]
        if missing:
            raise ValueError(
                "The colorDict parameter is missing values for the following labels : "
                + ", ".join(missing)
            )
    LOGGER.debug("The colordict value are : %s", colorDict)
    return colorDict


def _create_dataframe(
    left: Union[List, ndarray, Series],
    leftWeight: Union[ndarray, Series],
    right: Union[ndarray, Series],
    rightWeight: Union[ndarray, Series],
) -> DataFrame:
    # Create Dataframe
    if isinstance(left, pd.Series):
        left = left.reset_index(drop=True)
    if isinstance(right, pd.Series):
        right = right.reset_index(drop=True)
    if isinstance(leftWeight, pd.Series):
        leftWeight = leftWeight.reset_index(drop=True)
    if isinstance(rightWeight, pd.Series):
        rightWeight = rightWeight.reset_index(drop=True)
    data_frame = pd.DataFrame(
        {
            "left": left,
            "right": right,
            "leftWeight": leftWeight,
            "rightWeight": rightWeight,
        },
        index=range(len(left)),
    )
    if len(data_frame[(data_frame.left.isnull()) | (data_frame.right.isnull())]):
        raise NullsInFrame("Sankey graph does not support null values.")
    return data_frame


def plot_strips(
    ax: Any,
    colorDict: Union[Dict[str, Tuple[float, float, float]], Dict[str, str]],
    dataFrame: DataFrame,
    leftLabels: ndarray,
    leftWidths: Dict,
    ns_l: Dict,
    ns_r: Dict,
    rightColor: bool,
    rightLabels: ndarray,
    rightWidths: Dict,
    xMax: float64,
) -> None:
    # Plot strips
    for leftLabel in leftLabels:
        for rightLabel in rightLabels:
            label_color = leftLabel
            if rightColor:
                label_color = rightLabel
            if (
                len(
                    dataFrame[
                        (dataFrame.left == leftLabel) & (dataFrame.right == rightLabel)
                    ]
                )
                > 0
            ):
                # Create array of y values for each strip, half at left value,
                # half at right, convolve
                ys_d = np.array(
                    50 * [leftWidths[leftLabel]["bottom"]]
                    + 50 * [rightWidths[rightLabel]["bottom"]]
                )
                ys_d = np.convolve(ys_d, 0.05 * np.ones(20), mode="valid")
                ys_d = np.convolve(ys_d, 0.05 * np.ones(20), mode="valid")
                ys_u = np.array(
                    50 * [leftWidths[leftLabel]["bottom"] + ns_l[leftLabel][rightLabel]]
                    + 50
                    * [rightWidths[rightLabel]["bottom"] + ns_r[leftLabel][rightLabel]]
                )
                ys_u = np.convolve(ys_u, 0.05 * np.ones(20), mode="valid")
                ys_u = np.convolve(ys_u, 0.05 * np.ones(20), mode="valid")

                # Update bottom edges at each label so next strip starts at the
                # right place
                leftWidths[leftLabel]["bottom"] += ns_l[leftLabel][rightLabel]
                rightWidths[rightLabel]["bottom"] += ns_r[leftLabel][rightLabel]
                ax.fill_between(
                    np.linspace(0, xMax, len(ys_d)),
                    ys_d,
                    ys_u,
                    alpha=0.65,
                    color=colorDict[label_color],
                )
    ax.axis("off")


def _get_positions_and_total_widths(
    df: DataFrame, labels: ndarray, side: str
) -> Tuple[Dict, float64]:
    """Determine positions of label patches and total widths"""
    widths: Dict = defaultdict()
    for i, label in enumerate(labels):
        label_widths = {}
        label_widths[side] = df[df[side] == label][side + "Weight"].sum()
        if i == 0:
            label_widths["bottom"] = 0
            label_widths["top"] = label_widths[side]
        else:
            bottom_width = widths[labels[i - 1]]["top"]
            weighted_sum = 0.05 * df[side + "Weight"].sum()
            label_widths["bottom"] = bottom_width + weighted_sum
            label_widths["top"] = label_widths["bottom"] + label_widths[side]
            topEdge = label_widths["top"]
        widths[label] = label_widths
        LOGGER.debug("%s position of '%s' : %s", side, label, label_widths)
    return widths, topEdge
